In [53]:
%matplotlib widget
import pandas as pd
import numpy as np

from voxel import *
from mol_tools import *

import torch
import torch.nn as nn
import random
from tqdm import tqdm

In [2]:
df = pd.read_pickle("icsd_221_cp5_3_species.pkl")
df

,auid,aurl,compound,composition,species,natoms,spacegroup,pearson_symbol,geometry,positions_fractional,positions_cartesian,enthalpy_atom,enthalpy_formation_atom
0,aflow:c2bae074d76f81b6,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/CUB/Ag1F3...,Ag1F3Zn1,"[1, 3, 1]","[Ag, F, Zn]",5,221,cP5,"[3.979183, 3.979183, 3.979183, 90.0, 90.0, 90.0]","[[0.0, 0.0, 0.0], [0.5, 0.5, 0.0], [0.0, 0.5, ...","[[0.0, 0.0, 0.0], [1.98959, 1.98959, -0.0], [0...",-3.35725,None
1,aflow:c568d79d0d0e9fc8,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/CUB/Ag1Mn...,Ag1Mn3N1,"[1, 3, 1]","[Ag, Mn, N]",5,221,cP5,"[4.265445, 4.265445, 4.265445, 90.0, 90.0, 90.0]","[[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 2.13272, 2.13272], [2....",-5.99960,None
2,aflow:8944b3eb2c189cb7,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/CUB/Ag1Nb...,Ag1Nb1O3,"[1, 1, 3]","[Ag, Nb, O]",5,221,cP5,"[4.000268, 4.000268, 4.000268, 90.0, 90.0, 90.0]","[[0.5, 0.5, 0.5], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[[2.00013, 2.00013, 2.00013], [0.0, 0.0, 0.0],...",-6.76521,None
3,aflow:9da57f587b92d20f,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/CUB/Ag3I1...,Ag3I1S1,"[3, 1, 1]","[Ag, I, S]",5,221,cP5,"[5.0311, 5.0311, 5.0311, 90.0, 90.0, 90.0]","[[0.5, 0.5, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, ...","[[2.51555, 2.51555, 0.0], [-0.0, 2.51555, 2.51...",-1.61106,None
4,aflow:d43abd3deaeccc51,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/CUB/Al1Bi...,Al1Bi1O3,"[1, 1, 3]","[Al, Bi, O]",5,221,cP5,"[3.793297, 3.793297, 3.793297, 90.0, 90.0, 90.0]","[[0.5, 0.5, 0.5], [0.0, 0.0, 0.0], [0.0, 0.5, ...","[[1.89665, 1.89665, 1.89665], [0.0, 0.0, 0.0],...",-6.54116,-2.28122
...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,aflow:e2807ec9ea15f2f7,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/CUB/O3Sr1...,O3Sr1Zr1,"[3, 1, 1]","[O, Sr, Zr]",5,221,cP5,"[4.198551, 4.198551, 4.198551, 90.0, 90.0, 90.0]","[[0.5, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 0.0, ...","[[2.09928, -0.0, 0.0], [0.0, 2.09928, -0.0], [...",-8.21154,-3.46243
758,aflow:80368671f9d16cb2,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/CUB/O3Sr1...,O3Sr1Zr1,"[3, 1, 1]","[O, Sr, Zr]",5,221,cP5,"[4.401751, 4.401751, 4.401751, 90.0, 90.0, 90.0]","[[0.0, 0.0, 0.5], [0.5, 0.0, 0.0], [0.0, 0.5, ...","[[0.0, 0.0, 2.20088], [2.20088, 0.0, 0.0], [0....",-5.73222,-0.983113
759,aflow:d94ef5a92bb70b71,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/CUB/O3Sr1...,O3Sr1Zr1,"[3, 1, 1]","[O, Sr, Zr]",5,221,cP5,"[4.200047, 4.200047, 4.200047, 90.0, 90.0, 90.0]","[[0.5, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 0.0, ...","[[2.10002, 0.0, 0.0], [0.0, 2.10002, 0.0], [0....",-8.21154,-3.46244
760,aflow:32ccebcea1af1ec7,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/CUB/O3Sr1...,O3Sr1Zr1,"[3, 1, 1]","[O, Sr, Zr]",5,221,cP5,"[4.198544, 4.198544, 4.198544, 90.0, 90.0, 90.0]","[[0.5, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 0.0, ...","[[2.09927, 0.0, 0.0], [0.0, 2.09927, 0.0], [-0...",-8.21154,-3.46243


In [3]:
# C,D,H,W
def make_single_channel_descriptor(mol, sigma, L, N):
    descriptor = np.zeros((len(elements), N, N, N))
    
    A = calc_basis(mol.geometry)
    coords = calc_cartesian_positions(A, mol.positions_fractional)
    
    mx, my, mz = get_mesh_coords(A, L, N)
            
    B, G, SG = reciprocal_lattice_gaussian(A, coords, sigma, mx, my, mz)
    
    descriptor = adapt_to_voxel_grid(G, SG, L, N) # (N,N,N)
    
    return descriptor

In [89]:
def prepare_single_channel_data(df, sigma, L, N):
    df = df.drop_duplicates(subset="compound", keep="first")
    print("Number of molecules", df.shape[0])
    
    width = 2 * np.pi / L * N
    adj_width = width*np.sqrt(3)
    
    names = []
    reciprocal_data = []
    ys = []
    for index, mol in df.iterrows():
        A = calc_basis(mol.geometry)
        coords = calc_cartesian_positions(A, mol.positions_fractional)
        mx, my, mz = get_mesh_coords(A, adj_width)
        B, G, SG = reciprocal_lattice_gaussian(A, coords, sigma, mx, my, mz)

        names.append(mol.compound)
        reciprocal_data.append((G, SG))
        ys.append(mol.enthalpy_atom)
                    
    return names, reciprocal_data, ys
        

In [5]:
names, reciprocal_data, ys = prepare_single_channel_data(df, sigma=0.1, L=12.8, N=32)

In [6]:
len(names)

377

In [71]:
class MolLoader(object):
    def __init__(self, mol_df, L, N, batch_size,
                 shuffle=False, rotate_randomly=False, device=torch.device('cpu')):
        
        names, reciprocal_data, ys = prepare_single_channel_data(df, sigma=0.1, L=12.8, N=32)
        
        self.names = names
        self.reciprocal_data = reciprocal_data
        self.ys = ys
        self.batch_size = batch_size
        self.rotate_randomly = rotate_randomly
        self.device = device
        
        self.L = L
        self.N = N
        
        self.shuffle = shuffle
        self.current = 0
        self.indices = np.arange(len(ys))
        self.N_data = len(self.indices)
        
    def __iter__(self):
        return self
    
    def __next__(self):
         return self.next()

    def __len__(self):
        return int(np.ceil(self.N_data / self.batch_size))

    def next(self):
        N_data = self.N_data
        N = self.N # number of voxels
        L = self.L
        
        if self.current == 0 and self.shuffle:
            np.random.shuffle(self.indices)
        
        if self.current < N_data:
            n1 = self.current
            n2 = min(self.current + self.batch_size, N_data)
            
            x_names = []
            x = np.zeros((n2-n1, 1, N, N, N), dtype="float32")
            y = np.zeros((n2-n1,1), dtype="float32")
            
            for i, j in enumerate(range(n1, n2)):
                data_index = self.indices[j] # supports shuffling
                
                G, SG = self.reciprocal_data[data_index]
                y[i] = self.ys[data_index]
                x_names.append(self.names[data_index])
                
                R = np.eye(3)
                if self.rotate_randomly:
                    R = get_random_3D_rotation_matrix()
                
                descriptor = adapt_to_voxel_grid(G, SG, L, N, rot=R)
                # print(i)
                x[i, 0, :, :, :] = descriptor
            
            self.current = n2
            
            x = torch.from_numpy(x).to(self.device, non_blocking=True)
            y = torch.from_numpy(y).to(self.device, non_blocking=True)
            
            return (x_names, x, y)
        
        self.current = 0
        # print("Stop Iteration")
        raise StopIteration()
    

In [83]:
ml = MolLoader(df, L=12.8, N=32, batch_size=32,
               shuffle=False, rotate_randomly=True)

In [85]:
%%time
tx = None
ty = None
for i, (names, x, y) in enumerate(ml):
    tx = x
    ty = y
    print(i, names, x.shape, y.shape)

0 ['Al1N1Pr3', 'Al1N1Tb3', 'Al1N1Ti3', 'As1Ca3N1', 'Au1Ca3N1', 'Au1Cl3Cs1', 'Au1K3O1', 'Au1N1V3', 'Au1O1Rb3', 'B1Ce1Pd3', 'B1Ce1Rh3', 'B1Dy1Pd3', 'B1Dy1Rh3', 'B1Er1Pd3', 'B1Er1Rh3', 'B1Eu1Pd3', 'B1Eu1Rh3', 'B1Gd1Pd3', 'B1Gd1Rh3', 'B1Hf1Rh3', 'B1Ho1Pd3', 'B1Ho1Rh3', 'B1In1La3', 'B1In1Sc3', 'B1Ir3Sc1', 'B1La1Pd3', 'B1La1Rh3', 'B1Lu1Pd3', 'B1Lu1Rh3', 'B1Nd1Pd3', 'B1Nd1Rh3', 'B1Pb1Sc3'] torch.Size([32, 1, 32, 32, 32]) torch.Size([32, 1])
1 ['B1Pd3Pr1', 'B1Pd3Tb1', 'B1Pd3Yb1', 'B1Pr1Rh3', 'B1Rh3Sc1', 'B1Rh3Tb1', 'B1Rh3Tm1', 'B1Rh3U1', 'B1Rh3Y1', 'B1Rh3Yb1', 'B1Rh3Zr1', 'B1Sc3Sn1', 'B1Sc3Tl1', 'Ba1Ce1O3', 'Ba1F3Li1', 'Ba1Fe1O3', 'Ba1H3Li1', 'Ba1Ir1O3', 'Ba1Mo1O3', 'Ba1Nb1O3', 'Ba1O3Pa1', 'Ba1O3Pb1', 'Ba1O3Pr1', 'Ba1O3Pu1', 'Ba1O3Sn1', 'Ba1O3Tb1', 'Ba1O3Th1', 'Ba1O3Ti1', 'Ba1O3Zr1', 'Ba3O1Pb1', 'Ba3O1Sn1', 'Be1Cs1F3'] torch.Size([32, 1, 32, 32, 32]) torch.Size([32, 1])
2 ['Be1H3Na1', 'Bi1Ca3N1', 'Bi1Ga1O3', 'Bi1In1O3', 'Bi1N1Sr3', 'Bi1O3Sc1', 'Br1K3O1', 'Br1Li3O1', 'Br1O1Rb3', 'Br3Ca1Cs1', 'B

In [39]:
def seed_everything(seed=0):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)

In [40]:
def count_parameters(model):
    s = 0
    for t in model.parameters():
        s += np.prod(t.shape)
    return s

In [41]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        # print("CUDA version:", torch.version.cuda)
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [49]:
class VoxelNet(nn.Module):
    def __init__(self, in_channels=1):
        super().__init__()
        # (N, C, D, H, W)
        self.conv1 = nn.Sequential(
            nn.Conv3d(in_channels, 16, 3, padding=0, stride=1),
            nn.BatchNorm3d(16),
            nn.LeakyReLU(0.3),
            nn.MaxPool3d(2))
        
        self.conv2 = nn.Sequential(
            nn.Conv3d(16, 16, 3, padding=0, stride=1),
            nn.LeakyReLU(0.3),
            nn.MaxPool3d(2))
        
        self.conv3 = nn.Sequential(
            nn.Conv3d(16, 16, 3, padding=0, stride=1),
            nn.LeakyReLU(0.3))
        
        self.regressor = nn.Sequential(nn.Flatten(),
                                        nn.Linear(1024, 32),
                                        nn.LeakyReLU(0.3),
                                        nn.Linear(32, 8),
                                        nn.LeakyReLU(0.3),
                                        nn.Linear(8, 1))
    def forward(self, x):
        #print(x.shape)
        x = self.conv1(x)
        # print("conv1", x.shape)
        x = self.conv2(x)
        # print("conv2", x.shape)
        x = self.conv3(x)
        # print("conv3", x.shape)
        x = self.regressor(x)
        return x

In [43]:
net = VoxelNet()

In [44]:
count_parameters(net)

47409

In [45]:
net(tx)

torch.Size([32, 1, 32, 32, 32])


tensor([[0.0867],
        [0.0883],
        [0.0856],
        [0.0948],
        [0.0831],
        [0.0940],
        [0.0823],
        [0.0939],
        [0.0936],
        [0.0865],
        [0.0945],
        [0.0938],
        [0.0902],
        [0.0878],
        [0.0951],
        [0.0951],
        [0.0873],
        [0.0897],
        [0.0913],
        [0.0881],
        [0.0943],
        [0.0940],
        [0.0831],
        [0.0950],
        [0.0931],
        [0.0927],
        [0.0919],
        [0.0929],
        [0.0918],
        [0.0896],
        [0.0900],
        [0.0942]], grad_fn=<AddmmBackward>)

In [46]:
ty

tensor([[ -3.3572],
        [ -5.9996],
        [ -6.7652],
        [ -1.6111],
        [ -6.5412],
        [ -5.0429],
        [ -4.6094],
        [ -5.7439],
        [ -5.7647],
        [ -5.4209],
        [-11.2165],
        [ -5.7528],
        [ -5.5573],
        [ -6.6131],
        [ -6.5054],
        [ -5.9943],
        [ -4.6213],
        [ -6.2439],
        [ -3.9499],
        [ -6.0132],
        [ -3.5869],
        [ -6.8957],
        [ -7.5773],
        [ -6.1195],
        [ -6.0962],
        [ -6.0977],
        [ -9.5977],
        [ -6.0954],
        [ -7.8986],
        [ -5.9433],
        [ -6.1509],
        [ -4.9878]])

In [79]:
def fit(epochs, model, train_loader, opt, lr, weight_decay):
    t0 = time.time()
    
    optimizer = opt(model.parameters(), lr=lr, weight_decay=weight_decay)
    #print(optimizer)
    
    loss = nn.MSELoss()
    
    
    # best_val_acc = result["val_acc"]
    # best = deepcopy(model)
    
    for epoch in range(epochs):
        train_losses = []
        
        # Training Phase 
        model.train()
        for names, x, y in train_loader:
            y_hat = model(x)
            out = loss(y_hat, y)
            out.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            train_losses.append(out)
            
        # Validation phase
        # model.eval()
        #result = evaluate(model, val_loader, "val")
        total_loss = torch.stack(train_losses).mean().item()
        
        print(f"Epoch {epoch}: loss {total_loss}")
        
        
        # store model which performs best on validation set
        #if result["val_acc"] > best_val_acc:
        #    print("New best!")
        #    best_val_acc = result["val_acc"]
        #    best = deepcopy(model)
    
    t1 = time.time()
    print(f"Finished in {t1-t0:.4f}s.")

In [88]:
df = pd.read_pickle("icsd_221_cp5_3_species.pkl")

seed_everything()
device = get_default_device()

net = VoxelNet()
net = net.to(device)

ml = MolLoader(df, L=12.8, N=32, batch_size=32,
               shuffle=True, rotate_randomly=True, device=device)

fit(10, net, ml, torch.optim.Adam, lr=0.001, weight_decay=0)

Epoch 0: loss 12.705279350280762
Epoch 1: loss 3.346860408782959
Epoch 2: loss 2.79807448387146
Epoch 3: loss 2.6790273189544678
Epoch 4: loss 2.8494441509246826
Epoch 5: loss 2.8785037994384766
Epoch 6: loss 2.670045852661133
Epoch 7: loss 2.5662145614624023
Epoch 8: loss 2.531036853790283
Epoch 9: loss 2.6039369106292725
Finished in 425.7920s.


In [90]:
df = pd.read_pickle("all_221_cp5_3_species_oxides.pkl")

seed_everything()
device = get_default_device()

net = VoxelNet()
net = net.to(device)

ml = MolLoader(df, L=12.8, N=32, batch_size=64,
               shuffle=True, rotate_randomly=True, device=device)

fit(25, net, ml, torch.optim.Adam, lr=0.001, weight_decay=0)

Number of molecules 1983
Epoch 0: loss 5.580891132354736
Epoch 1: loss 2.226628303527832
Epoch 2: loss 2.145143985748291
Epoch 3: loss 2.1279728412628174
Epoch 4: loss 2.1690125465393066
Epoch 5: loss 2.1462810039520264
Epoch 6: loss 2.1388800144195557
Epoch 7: loss 2.138343572616577
Epoch 8: loss 2.2773029804229736
Epoch 9: loss 2.2712244987487793
Epoch 10: loss 2.168891191482544
Epoch 11: loss 2.210529327392578
Epoch 12: loss 2.1129002571105957
Epoch 13: loss 2.101436138153076
Epoch 14: loss 2.1485226154327393
Epoch 15: loss 2.1439483165740967
Epoch 16: loss 2.255344867706299
Epoch 17: loss 2.3504998683929443
Epoch 18: loss 2.1770009994506836
Epoch 19: loss 2.2940027713775635
Epoch 20: loss 2.113391637802124
Epoch 21: loss 2.176915407180786
Epoch 22: loss 2.1534571647644043
Epoch 23: loss 2.086427688598633
Epoch 24: loss 2.203551769256592
Finished in 3549.2296s.


2